In [ ]:
import numpy as np
import holoviews as hv; hv.extension('bokeh')
import panel as pn;     pn.extension()
from panel.interact import interact

from julia.api import Julia
jl = Julia(compiled_modules=False)
from julia import Main

%load_ext julia.magic

In [2]:
%%julia
using Pkg, Revise
gla_dir = "../GenLinAlgProblems"
Pkg.activate(gla_dir)
using GenLinAlgProblems, LinearAlgebra, RowEchelon, LaTeXStrings, Latexify, Markdown, Printf

using PyCall
py_display = pyimport("IPython.display").display
py_Latex   = pyimport("IPython.display").Latex

function py_show(args...; kwargs...)  py_display(py_Latex(l_show(args...; kwargs...))) end;

  Activating project at `~/elementary-linear-algebra/GenLinAlgProblems`


In [92]:
%%julia
using Random
function gen_nilpotent_matrix(n::Int; maxint=3, max_subdiags=n-1)
    P, P_inv = gen_inv_pb(n, maxint=maxint)
    if n > 5   # mitigate numerical inaccuracies in P_inv
        P     = Rational{Int}.(P)
        P_inv = inv(P)
    end
    num_subdiags = rand(1:max_subdiags)
    N = [ (i > j && i - j <= num_subdiags) ? rand(-maxint:maxint) : 0 for i in 1:n, j in 1:n ]
    round.( P * N * P_inv, digits=0)
end;

<div style="float:center;width:100%;text-align:center;">
<strong style="height:100px;color:darkred;font-size:40px;">Nilpotent Matrices and Their Applications</strong><br>
</div>

# 1. Introduction

## 1.1 Definition and Examples

<div style="float:left;width:100%;background-color:#F2F5A9;color:black;">

**Definition:** A matrix $A$ is **nilpotent** if there exists a positive integer $k$ such that $A^k = 0.$<br>
$\qquad$ The smallest such $k$ is called the **nilpotency index** of the matrix.
</div>

For example, the following matrix $A$ is nilpotent with a nilpotency index of 3:

In [75]:
%%julia
A  = [0 1 0 0; 0 0 1 0; 0 0 0 1; 0 0 0 0]
A2 = A^2
A3 = A2 * A
A4 = A3 * A

py_show( L"A =", A, L"\qquad A^2 =", A2, L"\qquad A^3 =", A3, L"\qquad A^4 =", A4 )

<IPython.core.display.Latex object>

____
Let us write an *is_nilpotent()* function and test it

In [76]:
%%julia
function is_nilpotent(matrix::AbstractMatrix, max_power::Int=size(matrix,1))
    # with larger integer matrices, this is likely to fail!
    powers        = [matrix]  # Initialize list to store intermediate powers
    current_power = matrix

    for k in 2:max_power
        current_power = current_power * matrix
        push!(powers, current_power)
        if norm(current_power) ≈ 0.0  # Check if the matrix is close to zero
            return (true, k, powers)
        end
    end

    return (false, nothing, powers)  # returns true/false, index of nilpotency, matrix powers
end;

In [85]:
%%julia
function tst_3x3(n=3)
    for _ in 1:n
        A = Int.(gen_nilpotent_matrix(3))
        ok,_ndx,m_list = is_nilpotent( A )

        powers = []
        for i in 1:_ndx  push!( powers, l_show(L"\quad A^{%$(i-1)} =", m_list[i])) end
        py_show(powers...)
    end
end
tst_3x3()


P * P_inv ≈ I = true


<IPython.core.display.Latex object>

P * P_inv ≈ I = true

<IPython.core.display.Latex object>


P * P_inv ≈ I = true


<IPython.core.display.Latex object>

In [152]:
def visualize_nilpotent_matrix(powers, ok):
    if powers is None: return
    n           = powers[0].shape[0]
    ndx         = len(powers)
    abs_powers  = [np.abs(power) for power in powers]
    # max_val     = max(*[np.max(abs_powers[k]) for k in range(ndx)],1e-6)
    max_val     = max(np.max(abs_powers[0]), 1e-6)
    epsilon     = max_val * 1e-3  # Small fraction of max_val to create a gap
    color_range = (-epsilon, max_val)  # Widen the range to ensure zero is distinct

    heatmaps = {
        k : hv.HeatMap((range(n), range(n), abs_powers[k]))\
                    .opts(xticks=None, yticks=None, title = f"A^{k}")
        for k in range(ndx)
    }

    dmap = hv.DynamicMap(
        lambda i: heatmaps[i], kdims=["Power"]
    ).redim.values(Power=list(range(0, ndx)))

    return dmap.opts( width=300, height=300, colorbar=True, cmap="plasma", clim=color_range,
        tools=["hover"]
    )

def heatmap_of_powers(N):
    maxint = 3 if N < 10 else 1
    ok,_,powers = Main.is_nilpotent( Main.gen_nilpotent_matrix(N, maxint=maxint),N+1)
    return powers,ok
powers,ok = heatmap_of_powers(15)
print(f"For larger matrices, the numerical computations may fail:\nOk numerically? {ok}")
visualize_nilpotent_matrix(powers,ok)

For larger matrices, the numerical computations may fail:
Ok numerically? True


:DynamicMap   [Power]
   :HeatMap   [x,y]   (z)

## 1.2 Eigenvalues and Eigenvectors

### 1.2.1 Eigenvalues

<div style="float:left;width:100%;height:0.8cm;background-color:#F2F5A9;color:black;">

**Theorem:** All **eigenvalues** of a nilpotent matrix are zero.
</div>

Consider a nilpotent matrix $A$ with nilpotency index $k$ and an eigenpair $(\lambda, v)$

$\qquad A v = \lambda v \Rightarrow  A^k v = \lambda^k v$:<br>
$\qquad$ Since $A^k = 0,\;\;$ we therefore have $\;\;\lambda^k v = 0 \Rightarrow \lambda^k v^t v = 0$.<br>
$\qquad$ Since $v \neq 0,\;\; v^t v = \Vert v \Vert^2 \ne 0\;\;\;\therefore \;\;\lambda = 0$.

#### Eigenvectors of Nilpotent Matrices

The eigenvectors of a nilpotent matrix \( A \) correspond to the eigenvalue \( \lambda = 0 \). However, not all vectors in the null space of \( A \) are eigenvectors. Some vectors may only become zero after repeated applications of \( A \).

To fully understand the behavior of a nilpotent matrix, we also consider its **generalized eigenvectors**, which satisfy:

\[
(A - \lambda I)^m \mathbf{v} = 0
\]

for some positive integer \( m \). For a nilpotent matrix \( A \), this simplifies to:

\[
A^m \mathbf{v} = 0.
\]

## 1.3 Importance of Nilpotent Matrices

Nilpotent matrices are a cornerstone of linear algebra, with significant theoretical and practical implications:

1. **Jordan Canonical Form**:
   - Nilpotent matrices form the building blocks of the Jordan canonical form, which represents any square matrix as a sum of a diagonalizable matrix and a nilpotent matrix. Understanding nilpotent matrices is crucial for analyzing the structure of linear operators.

2. **Differential Equations**:
   - When solving systems of differential equations, nilpotent matrices simplify the computation of matrix exponentials \( e^{tA} \). This is particularly useful for matrices with repeated eigenvalues.

3. **Iterative Algorithms**:
   - In numerical methods and iterative algorithms, nilpotent-like structures arise naturally, particularly in the context of Krylov subspaces.

4. **Decompositions**:
   - Many matrix factorizations, like the LU decomposition and singular value decomposition (SVD), involve submatrices that may exhibit nilpotent behavior.

5. **Theoretical Insights**:
   - The properties of nilpotent matrices (e.g., all eigenvalues are zero, trace and determinant are zero) reveal deeper insights into linear transformations and the behavior of square matrices.
  
Nilpotent matrices are important in various fields of mathematics and science, including:
- Understanding matrix decomposition and eigenvalues.
- Applications in differential equations and dynamical systems.
- Connections to Lie algebras and other abstract algebraic structures.

## Key Properties
1. For a nilpotent matrix \( A \), there exists a smallest positive integer \( k \) (called the **index of nilpotency**) such that \( A^k = 0 \).
2. All eigenvalues of a nilpotent matrix are zero.
3. A nilpotent matrix is always singular (non-invertible).
4. Nilpotent matrices play a key role in the Jordan canonical form.

# 5. Applications
## Application 1: Jordan Canonical Form

In [337]:
from scipy.linalg import jordan_form
J,P=jordan_form(Main.A)

ImportError: cannot import name 'jordan_form' from 'scipy.linalg' (/opt/conda/lib/python3.12/site-packages/scipy/linalg/__init__.py)

In [ ]:
%%julia
using LinearAlgebra, SymPy

# Function to compute Jordan canonical form
function jordan_form_example(A)
    F, J = jordan_form(Sym(A))
    return F, J
end

# Compute Jordan form for the nilpotent matrix A
A = gen_nilpotent_matrix(3)
F, J = jordan_form_example(A)
(F, J)


In [336]:
%%julia
jordan_form( Sym(A) )

TypeError: object of type 'Zero' has no len()

## Application 2: Solving Linear Systems

In [ ]:
%%julia
# Function to compute the matrix exponential
using LinearAlgebra

function matrix_exponential(A, t)
    return exp(t * A)
end

# Compute the exponential of A for t = 1.0
t = 1.0
exp_result = matrix_exponential(A, t)
exp_result

### Exercises
1. Prove that any strictly upper triangular matrix is nilpotent.
2. Compute the nilpotency index of the following matrix using Julia:
   \[
   C = \begin{bmatrix} 0 & 1 & 2 \\ 0 & 0 & 1 \\ 0 & 0 & 0 \end{bmatrix}
   \]
3. Write a Julia function to verify if a given matrix is nilpotent and determine its nilpotency index.
4. Use Julia to generate a random nilpotent matrix of size \( n = 5 \) and verify its properties.


## 7. Conclusion

### Conclusion
In this notebook, we explored nilpotent matrices using Julia for computations and Python for visualization. These matrices are fundamental in linear algebra and have significant applications in solving linear systems, understanding Jordan canonical forms, and iterative algorithms.

**Key Takeaways**:
- All eigenvalues of a nilpotent matrix are zero.
- Nilpotent matrices simplify computational tasks.
- Python and Julia together provide a powerful framework for exploring advanced linear algebra topics.

Continue experimenting with the exercises and exploring further applications!

In [9]:
%%julia
A=[1 2; 1 3]
Float64.(A)

array([[1., 2.],
       [1., 3.]])